In [1]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
        print("Setting GPU Memory Growth...")
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

import time
import datetime

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from plot_keras_history import plot_history
import sklearn.metrics as metrics
import os as os

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.python.client import device_lib

import kerastuner as kt
from kerastuner.tuners import RandomSearch, BayesianOptimization


print("TF version:", tf.__version__)
print("KT version:", kt.__version__)
print("Panda version:", pd.__version__)

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print("Physical devices: ", tf.config.list_physical_devices())
# Se va a habilitar la dedicacion dinamica de memoria para que la GPU vaya asignando recursos al proceso conforme los vaya necesitando

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
#print(tf.config.list_physical_devices('GPU'))
print("Build with CUDA: ", tf.test.is_built_with_cuda())

# Number of best models
numBestModels = 1 

# Validation set percentage
val_size = 0.3

# Batch size
batch = 1024

# Nodos internos
hidden_nodes = 10

# Units parameters
minUnits = 10
maxUnits = 50
stepsUnits = 10
defaultUnits = 25

# Layers parameters
minLayers = 1
maxLayers = 5
defaultLayers = 3

# Dropout Parameters
minDropout = 0
maxDropout = 0.33
defaultDropout = .25

# Establecer medida de loss
loss = "mean_absolute_error"

# Learning rate
learningRate = [0.0, 1e-2, 1e-3, 1e-4]

# Model metrics
modelMetrics = [keras.metrics.MAE, tf.keras.metrics.RootMeanSquaredError(name = 'rmse')]

# Optimizer objetive: error percetange with the validation set 
objective = kt.Objective('val_mean_absolute_error', 'min')

# Maximum model trials and executions
trials = 1
executions = 1

# Model saving parameters
workingDirectory = datetime.datetime.fromtimestamp(time.time()).strftime('%d-%m-%Y-%H:%M')
# projectName = "gru"

Setting GPU Memory Growth...
TF version: 2.1.0
KT version: 1.0.1
Panda version: 1.1.5
Physical devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Default GPU Device: /device:GPU:0
Num GPUs Available:  1
Build with CUDA:  True


In [17]:
# X = pd.read_csv("C:/Users/hecto/Documents/Master/TFM/tfm-renewable-energy-deep-learning/data/training/2015_PV_W20_H20.csv",
#                        delimiter=";")

# xTest = pd.read_csv("C:/Users/hecto/Documents/Master/TFM/tfm-renewable-energy-deep-learning/data/test/2016_PV_W20_H20.csv",
#                    delimiter=";")

X = pd.read_csv("C:/Users/hecto/Documents/Master/TFM/tfm-renewable-energy-deep-learning/data/training/2015_Weather_W2.csv",
                       delimiter=",")

xTest = pd.read_csv("C:/Users/hecto/Documents/Master/TFM/tfm-renewable-energy-deep-learning/data/test/2016_Weather_W2.csv",
                   delimiter=",")

print(X)
print(xTest)

     min_temp_d1  max_temp_d1  rainfall_d1  sun_hours_d1  max_wind_speed_d1  \
0       0.919431     0.699029     0.001095      0.774436           0.261538   
1       0.909953     0.621359     0.006572      0.315789           0.369231   
2       0.744076     0.684466     0.092004      0.902256           0.230769   
3       0.777251     0.582524     0.000000      0.285714           0.138462   
4       0.763033     0.650485     0.118291      0.909774           0.338462   
..           ...          ...          ...           ...                ...   
359     0.862559     0.733010     0.004381      0.060150           0.230769   
360     0.696682     0.626214     0.004381      0.383459           0.369231   
361     0.620853     0.504854     0.000000      0.616541           0.338462   
362     0.649289     0.665049     0.000000      0.872180           0.338462   
363     0.663507     0.669903     0.000000      0.857143           0.369231   

     temp_9_d1  Rel_hum_9_d1  cloud_cover_d1  wind_

     min_temp_d1  max_temp_d1  rainfall_d1  sun_hours_d1  max_wind_speed_d1  \
0       0.620853     0.679612     0.000000      0.894737           0.261538   
1       0.739336     0.577670     0.000000      0.067669           0.200000   
2       0.620853     0.519417     0.048193      0.037594           0.123077   
3       0.720379     0.597087     0.127054      0.000000           0.200000   
4       0.691943     0.771845     0.082147      1.000000           0.092308   
..           ...          ...          ...           ...                ...   
360     0.639810     0.621359     0.007667      0.917293           0.261538   
361     0.644550     0.665049     0.001095      0.932331           0.307692   
362     0.748815     0.713592     0.000000      0.947368           0.261538   
363     0.725118     0.791262     0.000000      0.894737           0.307692   
364     0.905213     0.864078     0.000000      0.984962           0.307692   

     temp_9_d1  Rel_hum_9_d1  cloud_cover_d1  wind_

In [18]:
# Epochs
epchs = 1000

W=20
H=20
PV_index = 14

# Number of measurements to predict
numPredictions = 1

projectName = "lstm30min2016-1kepochs-rs"
# projectName = "lstm30min2016-1kepochs-bo"

In [20]:
# Preprocessing for W type files:

# Filter for PV data
X = X.iloc[:, ::PV_index]
xTest = xTest.iloc[:, ::PV_index]

# Reset column names
new_column_names = [f'PV_{i+1}' for i in range(len(X.columns))]
X.columns = new_column_names
xTest.columns = new_column_names

Y = pd.DataFrame.copy(X)
yTest = pd.DataFrame.copy(xTest)

# Create Y from taking next day data
Y = Y.drop(0)
Y = Y.reset_index(drop = True)
Y.loc[len(Y)] = np.zeros(numPredictions)

yTest = yTest.drop(0)
yTest = yTest.reset_index(drop = True)
yTest.loc[len(yTest)] = np.zeros(numPredictions)

print(X)
print(Y)

         PV_1
0    0.919431
1    0.909953
2    0.744076
3    0.777251
4    0.763033
..        ...
359  0.862559
360  0.696682
361  0.620853
362  0.649289
363  0.663507

[364 rows x 1 columns]
         PV_1
0    0.909953
1    0.744076
2    0.777251
3    0.763033
4    0.720379
..        ...
359  0.696682
360  0.620853
361  0.649289
362  0.663507
363  0.000000

[364 rows x 1 columns]


In [ ]:
# Data preprocessing: each row will contain the 20 measures for each day , and the 10 measures for the following day
# X = pd.DataFrame(np.array(training).reshape(-1, W))
Y = pd.DataFrame.copy(X)
yTest = pd.DataFrame.copy(xTest)

Y.columns = ["col_{}".format(i) for i in range(W+1, (2*W)+1)]
Y = Y.drop(0)
Y = Y.reset_index(drop = True)
Y.loc[len(Y)] = np.zeros(numPredictions)

# Last row is deleted because it is the one used for the real prediction, 
# it is not useful for the training of the model. 
X.drop(X.tail(1).index, inplace = True)
Y.drop(Y.tail(1).index, inplace = True)

print("X Preproccessed shape: ", X.shape)
print("Y Preproccessed shape: ", Y.shape)
print("---------------------------------------------")

#  Uncomment in order to normalize data
# Data Normalization
# scaler = MinMaxScaler()
# X = pd.DataFrame(scaler.fit_transform(X), columns=["col_{}".format(i) for i in range(1, 21)])
# Y = pd.DataFrame(scaler.fit_transform(Y), columns=["col_{}".format(i) for i in range(21, 41)])


In [ ]:
dfPreproccessed = pd.concat([X, Y], axis = 1)

print("DataFrame Preproccessed:")
print(dfPreproccessed)
print("---------------------------------------------")

# Split the data into training and validation sets
xTrain, xVal, yTrain, yVal = train_test_split(X, 
                                            Y, 
                                            test_size = val_size, 
                                            random_state = 0, 
                                            shuffle=False)

# Show subsets shapes
print("Shapes:")
print("xTrain:\t"+str(xTrain.shape))
print("yTrain:\t"+str(yTrain.shape))
print("xVal:\t"+str(xVal.shape))
print("yVal:\t"+str(yVal.shape))
print("xTest:\t"+str(xTest.shape))
print("yTest:\t"+str(yTest.shape))
print("---------------------------------------------")

In [ ]:

# Model definition:
with tf.device('/gpu:0'): 
    
    def build_model(hp): 

        model = Sequential()
        
        # First layer: GRU. Number of units optimization
        model.add(GRU(units = hp.Int('units', 
                                    min_value = minUnits,
                                    max_value = maxUnits, 
                                    step = stepsUnits,
                                    default = defaultUnits),
                        return_sequences = False, 
                        input_shape = (xTrain.shape[1], 1)))

        # Second layer: Dropout with percetange optimization
        model.add(
            Dropout(hp.Float('dropout', 
                            min_value = minDropout,
                            max_value = maxDropout,
                            default = defaultDropout)))
        
        # Last layer:
        model.add(Dense(numPredictions))

        print("Summary: ")
        model.summary()

        # # Set Epsilon to 1, in order to fix huge MAPE values.
        # keras.backend.set_epsilon(1)

        # Model compilation: learning rate optimization
        model.compile(loss = loss,
                    optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', 
                                                                values = learningRate)),
                    metrics = metrics = modelMetrics)

        return model

In [ ]:
# Model training:
with tf.device('/gpu:0'): 
    # Create optimizer
    gruOptimizer = RandomSearch(build_model, 
                                objective = objective,
                                max_trials = trials, 
                                executions_per_trial = executions,
                                project_name = projectName,
                                overwrite=True)
    # Create optimizer using Bayesian Optimization 
    # lstmOptimizer = BayesianOptimization(build_model, 
    #                             objective = objective,
    #                             max_trials = trials, 
    #                             executions_per_trial = executions,
    #                             project_name = projectName,
    #                             overwrite=True)
    
    # Search space summary
    gruOptimizer.search_space_summary()

    # Search execution with epochs
    initialTime = time.time()
    gruOptimizer.search(x = np.reshape(xTrain.values, (xTrain.shape[0], xTrain.shape[1], 1)),
                        y = yTrain, 
                        epochs = epchs,
                        batch_size = batch,
                        validation_data=(np.reshape(xVal.values, (xVal.shape[0], xVal.shape[1], 1)), yVal))
    finalTime = time.time()

    # Results summary
    gruOptimizer.results_summary()

    # Get best model generated
    model = gruOptimizer.get_best_models(num_models = numBestModels)[0]

    # Model training with epochs 
    history = model.fit(
                np.reshape(xTrain.values, (xTrain.shape[0], xTrain.shape[1], 1)),
                yTrain,
                epochs = epchs,
                batch_size = batch,
                validation_data=(np.reshape(xVal.values, (xVal.shape[0], xVal.shape[1], 1)), yVal))

    # Show model info
    print("Tiempo de entrenamiento (en segundos):\t"+str(finalTime - initialTime))
    print("Tiempo de entrenamiento (en horas):\t"+str((finalTime - initialTime)/3600))
    print(history.history.keys())
    print(model.history.history)
    print(model.history.params)

In [ ]:
# Training and Validation loss curves
plt.plot(history.history['loss'], label = 'Training loss')
plt.plot(history.history['val_loss'], label = 'Validation loss')
plt.legend()
plt.show()

# Model evaluation with validation data
score = model.evaluate(np.reshape(xTest.values,(xTest.shape[0], xTest.shape[1], 1)), yTest)
print('Score:', score)